In [1]:
import heapq
from collections import namedtuple
from tqdm.auto import tqdm
import numpy as np
from random import choice

In [ ]:
# Configuration
PUZZLE_DIM = 4
RANDOMIZE_STEPS = 10000
action = namedtuple('Action', ['pos1', 'pos2'])

In [3]:
# Define goal state and helpers
GOAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
GOAL_POSITIONS = {value: divmod(idx, PUZZLE_DIM) for idx, value in enumerate(GOAL_STATE.flatten())}




In [4]:
def available_actions(state: np.ndarray, blank_pos: tuple[int, int]) -> list['Action']:
    x, y = blank_pos
    actions = []
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

# Perform an action
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def state_to_bytes(state: np.ndarray) -> bytes:
    return state.flatten().tobytes()

def bytes_to_state(state_bytes: bytes) -> np.ndarray:
    return np.frombuffer(state_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))


In [5]:
# Optimized heuristic with linear conflict
def weighted_manhattan_with_linear_conflict(state: np.ndarray, weights=(1.5, 1.0)) -> float:
    distance = 0
    linear_conflict = 0

    for idx, value in enumerate(state.flatten()):
        if value == 0:
            continue
        current_pos = divmod(idx, PUZZLE_DIM)
        goal_pos = GOAL_POSITIONS[value]
        tile_weight = weights[1] if value < PUZZLE_DIM**2 // 2 else weights[0]
        distance += tile_weight * (abs(current_pos[0] - goal_pos[0]) + abs(current_pos[1] - goal_pos[1]))

        # Linear conflict
        if current_pos[0] == goal_pos[0]:  # Same row
            for col in range(PUZZLE_DIM):
                neighbor = state[current_pos[0], col]
                if neighbor != 0 and neighbor != value:
                    neighbor_goal = GOAL_POSITIONS[neighbor]
                    if neighbor_goal[0] == current_pos[0] and neighbor_goal[1] < goal_pos[1] and col > current_pos[1]:
                        linear_conflict += 1
        if current_pos[1] == goal_pos[1]:  # Same column
            for row in range(PUZZLE_DIM):
                neighbor = state[row, current_pos[1]]
                if neighbor != 0 and neighbor != value:
                    neighbor_goal = GOAL_POSITIONS[neighbor]
                    if neighbor_goal[1] == current_pos[1] and neighbor_goal[0] < goal_pos[0] and row > current_pos[0]:
                        linear_conflict += 1

    return distance + 2 * linear_conflict

def ida_star_solver(start_state: np.ndarray):
    def search(path, g_score, threshold):
        """
        Perform depth-first search up to a given cost threshold.
        """
        current_state_np, blank_pos, f_score = path[-1]
        if f_score > threshold:
            return f_score, None

        # Goal state check
        if current_state_np.tobytes() == GOAL_STATE.tobytes():
            return f_score, path

        min_cost = float('inf')
        for action in available_actions(current_state_np, blank_pos):
            new_state_np = do_action(current_state_np, action)
            new_blank_pos = action.pos2

            # Avoid revisiting states in the current path
            if any((new_state_np == state).all() for state, _, _ in path):
                continue

            h_score = weighted_manhattan_with_linear_conflict(new_state_np)
            new_f_score = g_score + 1 + h_score

            # Recurse
            result, solution_path = search(path + [(new_state_np, new_blank_pos, new_f_score)], g_score + 1, threshold)
            if solution_path is not None:
                return result, solution_path

            min_cost = min(min_cost, result)

        return min_cost, None

    # Initial heuristic and cost threshold
    blank_idx = start_state.flatten().tolist().index(0)
    blank_pos = divmod(blank_idx, PUZZLE_DIM)
    h_score = weighted_manhattan_with_linear_conflict(start_state)
    threshold = h_score
    path = [(start_state, blank_pos, h_score)]

    total_actions_evaluated = 0
    while True:
        total_actions_evaluated += 1
        result, solution_path = search(path, 0, threshold)
        if solution_path is not None:
            return solution_path, total_actions_evaluated
        if result == float('inf'):
            return None, total_actions_evaluated
        threshold = result


        
def reconstruct_path(parents, state_hash):
    path = []
    while state_hash is not None:
        path.append(state_hash)
        state_hash, state_bytes = parents[state_hash]
    return path[::-1], state_bytes


In [6]:
# Randomization for start state
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
blank_pos = tuple(map(int, np.where(state == 0)))

for r in tqdm(range(RANDOMIZE_STEPS), desc="Randomizing"):
    actions = available_actions(state, blank_pos)
    chosen_action = choice(actions)
    state = do_action(state, chosen_action)
    blank_pos = chosen_action.pos2

# Solve
print("Start State:\n", state)
solution_path, total_actions_evaluated = ida_star_solver(state)

if solution_path:
    num_actions = len(solution_path) - 1
    efficiency = num_actions / total_actions_evaluated
    print(f"Solution found in {num_actions} moves!")
    print("Final state after solving:\n", solution_path[-1][0])
    print(f"Number of actions on solution: {num_actions}")
    print(f"Total actions evaluated: {total_actions_evaluated}")
    print(f"Efficiency: {efficiency:.4f}")
else:
    print("No solution exists.")

C:\Users\matti\AppData\Local\Temp\ipykernel_18424\2053727506.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  blank_pos = tuple(map(int, np.where(state == 0)))


Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

Start State:
 [[12  4  9  5]
 [ 8  7  6 10]
 [15 11  1 14]
 [ 3  0  2 13]]
Solution found in 68 moves!
Final state after solving:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
Number of actions on solution: 68
Total actions evaluated: 13
Efficiency: 5.2308
